<a href="https://colab.research.google.com/github/AmandinChyba/Project1-Fields-2022/blob/main/BIEM_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import jax

import matplotlib.pyplot as plt

In [17]:
# parametrized boundary
def boundary(t):
  x = (1 + np.sin(10*np.pi*t)/4) * np.cos(2*np.pi*t)
  y = (1 + np.sin(10*np.pi*t)/4) * np.sin(2*np.pi*t)
  return x,y

# boundary condition
def boundaryCond(x,y):
  return np.power(x,2) - np.power(y,2)

# normal vector
def differentiate(x,y):
  dx = np.gradient(x)
  dy = np.gradient(y)
  #normal = np.divide(-1, np.array([dx,dy]))
  normal = np.array([-dy, dx])
  d2 = np.gradient(np.array([dx,dy]))
  return normal, d2[1]

In [21]:
# number of points
K = 10

# list of points
t = np.arange(K)/(K-1)
x,y = boundary(t)

# spacing of t
h = (t[-1] - t[0])/(K-1)

# get the normal and second derivative of the points on the boundary
normal, d2 = differentiate(x,y)
xNormal = normal[0]
yNormal = normal[1]

# create the "jump" identity matrix
I = np.diag([-0.5]*K)

# calculate the integration kernel
i = np.arange(K).reshape((1,K))
j = i.transpose()
K = h * (xNormal[i] * (x[j]-x[i]) + yNormal[i] * (y[j]-y[i])) / (2 * np.pi * (np.power(x[j]-x[i], 2) + np.power(y[j]-y[i], 2)))
K = np.nan_to_num(K, 0)

# solve for the charge density
a = I+K
b = boundaryCond(x,y)
soln = np.linalg.solve(a, b)
print('solution',soln)

solution [ 3.54682380e-14 -2.44307231e-01  2.52187333e+00  6.29941428e-01
 -2.33957713e+00 -8.29475038e-01  1.49070935e+00  1.34237069e+00
 -3.97908823e-01 -4.99409971e-14]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
